In [1]:
from numpy.random import seed
seed(1)

In [2]:
import numpy as np
import pandas as pd

import warnings
warnings.simplefilter('ignore', FutureWarning)

In [3]:
anxiety = pd.read_csv('cleaned_disorder.csv')

In [4]:
anxiety = anxiety.drop(['Unnamed: 0'],axis=1)

In [5]:
anxiety.columns

Index(['Sex', 'Race', 'Age', 'Number of Bio. Parents', 'Number of Siblings',
       'Poverty Status', 'Social Phobia', 'ADHD', 'CD', 'Depression', 'ODD',
       'Number of Impairments', 'Number of Type A Stressors',
       'Number of Type B Stressors', 'Frequency Temper Tantrums',
       'Frequency Irritable Mood', 'Number of Sleep Disturbances',
       'Number of Physical Symptoms', 'Number of Sensory Sensitivities',
       'Family History - Substance Abuse',
       'Family History - Psychiatric Diagnosis', 'disorder'],
      dtype='object')

In [6]:
anxiety.head()

,Sex,Race,Age,Number of Bio. Parents,Number of Siblings,Poverty Status,Social Phobia,ADHD,CD,Depression,...,Number of Type A Stressors,Number of Type B Stressors,Frequency Temper Tantrums,Frequency Irritable Mood,Number of Sleep Disturbances,Number of Physical Symptoms,Number of Sensory Sensitivities,Family History - Substance Abuse,Family History - Psychiatric Diagnosis,disorder
0,1,0,2,1,1,0,1,0,0,1,...,1,1,27,396,4,3,2,0,0,GAD
1,0,1,2,0,3,1,0,1,0,0,...,3,2,282,1179,6,1,0,1,1,None
2,1,1,3,2,1,0,0,0,0,0,...,1,0,3,0,9,4,1,0,0,SAD
3,1,0,4,2,0,0,0,0,0,0,...,2,0,30,45,6,0,0,0,1,None
4,1,0,5,2,3,0,1,0,1,1,...,3,8,637,1170,10,5,2,0,0,Both


In [7]:
X = anxiety.drop(["disorder"], axis=1)
y = anxiety["disorder"]
print(X.shape, y.shape)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1, stratify=y)
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)




(287, 21) (287,)


In [8]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=21))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=4, activation='softmax'))

In [10]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=0
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [12]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

72/72 - 0s - loss: 0.9272 - accuracy: 0.6667
Normal Neural Network - Loss: 0.927231232325236, Accuracy: 0.6666666865348816


In [13]:
# Save the model
model.save("voice_model_trained.h5")

In [14]:
 # Load the model
from tensorflow.keras.models import load_model
voice_model = load_model("voice_model_trained.h5")

In [15]:
model_loss, model_accuracy = voice_model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

72/72 - 0s - loss: 0.9272 - accuracy: 0.6667
Normal Neural Network - Loss: 0.927231232325236, Accuracy: 0.6666666865348816
